Essse notebook propõe a criação de dois classificador de sentimentos em textos, um deles criado por meio de Word2Vec e o outro utilizando redes neurais CNN.


##Sobre a base de dados
A base de dados utilizada é um conjunto de texto de textos com opiniões de consumidores sobre os produtos comprados em e-commerces, ela se encontra disponivel no [Kaggle](https://www.kaggle.com/datasets/fredericods/ptbr-sentiment-analysis-datasets).

Essa base é composta de vários arquivos *CSV* e com excessão do arquivo *concatenated.csv*, que é a concatenação de todos os outros arquivos da base, cada arquivo representa os dados de um e-commerce diferente,. Todos os arquivos csv possuem as mesmas colunas, das colunas existem as seguintes vão ser utilizadas no notebook.

* review_text, o texto com avaliação do consumidor sobre o produto
* polarity, se avaliação é favoravel tem valor 1 senão tem valor 0.

## Rede neural CNN para análise de textos

Apesar de criadas com objetivo original de analisar imagens as redes neurais CNN mostram-se uteis para *NLP* (análise de textos)  conforme dizem os livros *Deep Learning for Natural Language Processing* (RAAIJMAKERS, 2019) e *Deep Learning for NLP and Speech Recognition* (UDAY KAMATH et al., 2019).

# Download e carregamento do modelo word2vec

In [ ]:
!mkdir models
!mkdir models/word2vec
!curl -O http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s300.zip cbow_s300.zip
!mv cbow_s300.zip models/word2vec
!cd models/word2vec ; unzip -o -e  cbow_s300.zip

mkdir: não foi possível criar o diretório “models”: Arquivo existe
mkdir: não foi possível criar o diretório “models/word2vec”: Arquivo existe
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  886M  100  886M    0     0  11.1M      0  0:01:19  0:01:19 --:--:-- 11.2M
curl: (6) Could not resolve host: cbow_s300.zip
Archive:  cbow_s300.zip
  inflating: cbow_s300.txt           


In [ ]:
import gensim
import os
def load_word2vec_model():
    if(os.path.isfile('models/word2vec/cbow_s300')==False):
        model=gensim.models.KeyedVectors.load_word2vec_format('models/word2vec/cbow_s300.txt')
        model.save('models/word2vec/cbow_s300')
    model_word2vec=gensim.models.KeyedVectors.load('models/word2vec/cbow_s300')
    return model_word2vec

model_word2vec=load_word2vec_model()

# Carregamento da base de dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!mkdir data
!cp /content/drive/MyDrive/Colab\ Notebooks/braz_port_sent_ana_data/archive.zip data
!cd data ; unzip -o -e archive.zip

Archive:  archive.zip
  inflating: b2w.csv                 
  inflating: buscape.csv             
  inflating: concatenated.csv        
  inflating: olist.csv               
  inflating: utlc_apps.csv           
  inflating: utlc_movies.csv         


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
def get_dataset():
    df=pd.read_csv("data/concatenated.csv")
    df=df[['review_text','polarity']].dropna()
    data = list(zip(df['review_text'], df['polarity']))
    dataset={ 'all_dataset':data,'max_rating':df['polarity'].max()}
    return dataset

def get_dataset_of_train_and_test():
    data=get_dataset()
    train, test = train_test_split(data['all_dataset'], test_size=0.3, random_state=0)
    dataset_train_test={'train':train,'test':test,'max_rating':data['max_rating']}
    return dataset_train_test


In [ ]:
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql.functions import col, udf
import pyspark.sql.functions as F
from pyspark.sql.functions import expr
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.tree import RandomForest
from pyspark.ml.regression import RandomForestRegressor
import os 
os.environ["SPARK_LOCAL_DIRS"]='spark_temp'

MAX_MEMORY = "10g"

spark = SparkSession.builder.appName('sparkdf341')\
        .config("spark.executor.memory", MAX_MEMORY)\
        .config("spark.driver.memory", MAX_MEMORY)\
        .config("spark.local.dir",os.environ["SPARK_LOCAL_DIRS"])\
        .getOrCreate()

#spark.sparkContext.setLogLevel('ERROR')


23/01/16 15:54:18 WARN Utils: Your hostname, DESKTOP-DHUKHBA resolves to a loopback address: 127.0.1.1; using 172.31.103.234 instead (on interface eth0)
23/01/16 15:54:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/16 15:54:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/16 15:54:19 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


# Criar um classificador de sentimentos com word2vec

## Criar um dataset com as colunas word2vec (um vetor com o word2vec sum) e a coluna polarity 

In [ ]:
import numpy as np
from functools import partial
import shutil
import pathlib
import tqdm
import pandas as pd
def text_to_word2vec_sum(wor2vec_model,text):
    ncolumns=wor2vec_model['ola'].shape[0]
    word_vec=np.zeros(ncolumns,dtype=np.float)
    words=gensim.utils.simple_preprocess(str(text),min_len=1,max_len=25, deacc=False)
    for i in range(len(words)):
        word=words[i]
        if(word in wor2vec_model):
            word_vec=wor2vec_model[word]+word_vec
    return word_vec.tolist()


dataset=get_dataset()

def apply_wordtovec_to_dataset(dataset,model_word2vec):
    shutil.rmtree("data/dataset_word2vec",ignore_errors=True)
    pathlib.Path("data/dataset_word2vec").mkdir(parents=True, exist_ok=True)
    buffer_to_save_df_in_dict={'polarity':[],'word2vec':[]}
    number_of_subdf=0
    for row in tqdm.tqdm(dataset):
        array=text_to_word2vec_sum(model_word2vec,row[0])
        buffer_to_save_df_in_dict['word2vec'].append(array)
        buffer_to_save_df_in_dict['polarity'].append(row[1])
        if(len(buffer_to_save_df_in_dict['polarity'])>1000):
            sub_df_name="data/dataset_word2vec/"+str(number_of_subdf)+'.parquet'
            pd.DataFrame.from_dict(buffer_to_save_df_in_dict).to_parquet(sub_df_name)
            number_of_subdf=number_of_subdf+1
            buffer_to_save_df_in_dict={'polarity':[],'word2vec':[]}

    if(len(buffer_to_save_df_in_dict['polarity'])>0):
        sub_df_name="data/dataset_word2vec/"+str(number_of_subdf)+'.parquet'
        pd.DataFrame.from_dict(buffer_to_save_df_in_dict).to_parquet(sub_df_name)
            

apply_wordtovec_to_dataset(dataset['all_dataset'],model_word2vec)


#text_to_word2vec_sum_with_model=partial(text_to_word2vec_sum,model_word2vec)
#dataget_dataset()

/tmp/ipykernel_10200/3657524042.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("data/concatenated.csv")
  0%|                                               | 0/2386163 [00:00<?, ?it/s]/tmp/ipykernel_10200/1069106159.py:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  word_vec=np.zeros(ncolumns,dtype=np.float)
  0%|                                   | 985/2386163 [00:00<04:02, 9845.71it/s]/tmp/ipykernel_10200/1069106159.py:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior

Carregar o dataset word2vec

In [ ]:
df_word2vec=spark.read.parquet('data/dataset_word2vec')

## Criar um classificador de sentimentos para o dataset word2vec
### Utilizar o PyCaret para decidir qual o melhor algoritmo de classificação
Carregar uma amostra da base word2vec para um dataframe Pandas

In [ ]:
import tqdm
def pass_spark_df_word2vec_to_pandas_df(spark_df):
    df_with_dataset_word2vec_in_dict={'polarity':[]}
    firstIteration=True
    with tqdm.tqdm(total=spark_df.count()) as pbar:
        for iterator in spark_df.collect():
            word2vec=iterator['word2vec']
            if(firstIteration):
                for i in range(len(word2vec)):
                    df_with_dataset_word2vec_in_dict['col'+str(i)]=[]
                firstIteration=False
        
            for i in range(len(word2vec)):
                df_with_dataset_word2vec_in_dict['col'+str(i)].append(word2vec[i])
            df_with_dataset_word2vec_in_dict['polarity'].append(iterator['polarity'])
            pbar.update(1)
    return pd.DataFrame.from_dict(df_with_dataset_word2vec_in_dict)

df_word2vec_pd=pass_spark_df_word2vec_to_pandas_df(df_word2vec.sample(0.04))
df_word2vec_pd

100%|██████████| 95898/95898 [02:45<00:00, 580.91it/s] 


,polarity,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,col31,col32,col33,col34,col35,col36,col37,col38,col39,col40,col41,col42,col43,col44,col45,col46,col47,col48,col49,col50,col51,col52,col53,col54,col55,col56,col57,col58,col59,col60,col61,col62,col63,col64,col65,col66,col67,col68,col69,col70,col71,col72,col73,col74,col75,col76,col77,col78,col79,col80,col81,col82,col83,col84,col85,col86,col87,col88,col89,col90,col91,col92,col93,col94,col95,col96,col97,col98,col99,col100,col101,col102,col103,col104,col105,col106,col107,col108,col109,col110,col111,col112,col113,col114,col115,col116,col117,col118,col119,col120,col121,col122,col123,col124,col125,col126,col127,col128,col129,col130,col131,col132,col133,col134,col135,col136,col137,col138,col139,col140,col141,col142,col143,col144,col145,col146,col147,col148,col149,col150,col151,col152,col153,col154,col155,col156,col157,col158,col159,col160,col161,col162,col163,col164,col165,col166,col167,col168,col169,col170,col171,col172,col173,col174,col175,col176,col177,col178,col179,col180,col181,col182,col183,col184,col185,col186,col187,col188,col189,col190,col191,col192,col193,col194,col195,col196,col197,col198,col199,col200,col201,col202,col203,col204,col205,col206,col207,col208,col209,col210,col211,col212,col213,col214,col215,col216,col217,col218,col219,col220,col221,col222,col223,col224,col225,col226,col227,col228,col229,col230,col231,col232,col233,col234,col235,col236,col237,col238,col239,col240,col241,col242,col243,col244,col245,col246,col247,col248,col249,col250,col251,col252,col253,col254,col255,col256,col257,col258,col259,col260,col261,col262,col263,col264,col265,col266,col267,col268,col269,col270,col271,col272,col273,col274,col275,col276,col277,col278,col279,col280,col281,col282,col283,col284,col285,col286,col287,col288,col289,col290,col291,col292,col293,col294,col295,col296,col297,col298,col299
0,1.0,0.011598,-0.001446,0.318318,0.274571,0.503673,0.038363,0.363702,0.459251,-0.323013,0.300260,-0.566472,-0.551126,0.029326,0.070197,-0.254547,0.046126,-0.498161,-0.567802,-0.398036,0.198723,0.090257,-0.227334,0.034458,0.511276,0.420178,-0.645683,0.402585,0.110121,0.513231,-0.132175,-0.282195,-0.542631,0.365867,-0.093802,-0.413382,0.040277,-0.070466,0.557294,0.168455,0.097859,-0.051531,-0.365370,-0.596402,-0.123320,0.131340,0.041462,0.918274,-0.353518,0.127068,-0.050949,0.281119,-0.559844,0.510204,-0.016106,0.240491,-0.173616,0.081235,-0.398484,-0.381408,-0.130063,-0.088998,-0.218557,0.102467,0.660904,-0.378299,0.036035,0.463312,-0.333293,-0.017018,0.138290,-0.461741,-0.325970,0.121187,0.187241,-0.048983,0.322814,-0.522342,0.554963,-0.364116,-0.324031,-0.528240,0.781413,0.725521,0.085383,-0.299428,0.212778,-0.169142,-0.246773,0.019862,-0.193509,0.504851,0.220584,-0.390312,-0.014323,-0.274922,-0.448908,0.214425,0.167493,-0.072264,-0.022042,-0.000635,0.535645,0.029661,-0.489160,0.128887,-0.492210,-0.423378,-0.533628,-0.534781,-0.203499,0.109434,0.602510,0.642949,0.004485,0.183843,-0.717129,-0.220582,0.074219,0.325744,0.117135,-0.581116,-0.108978,0.051044,-0.272577,-0.491328,-0.015817,0.283717,0.330653,-0.181389,0.198133,-0.237636,0.358899,0.101739,0.164324,0.158847,-0.241729,-0.305446,0.476750,0.233260,0.464729,0.195817,-0.110283,-0.140629,0.042909,0.212700,-0.263472,0.150127,0.334158,-0.000706,0.227624,0.473773,0.304461,0.024249,-0.294935,0.256546,0.014061,0.176322,-0.312563,-0.672809,0.237728,0.222698,-0.271661,-0.396483,-0.267366,-0.122366,0.870902,-0.345506,-0.329951,-0.004829,-0.273216,0.189036,0.163761,-0.085962,-0.477202,0.116978,0.336111,0.079081,0.171003,-0.173582,-0.544650,-0.029398,-0.245162,-0.233786,0.025020,0.278083,-0.789171,-0.292956,-0.338798,0.042791,-0.136409,0.231094,0.360742,0.484045,-0.249625,-0.117833,0.147874,0.026096,0.179536,-0.094409,-0.446034,-0.103730,-0.017792,0.310521,-0.115763,-0.501520,-0.106928,-0.007473,0.355834,-0.441781,-0.05669

Aplicar o PyCaret na amostra 

In [ ]:
from pycaret.classification import *
clf1 = setup(data = df_word2vec_pd, target = 'polarity')
compare_models(exclude=['knn','gbc'])

,Description,Value
0,Session id,8704
1,Target,polarity
2,Target type,Binary
3,Original data shape,"(95898, 301)"
4,Transformed data shape,"(95898, 301)"
5,Transformed train set shape,"(67128, 301)"
6,Transformed test set shape,"(28770, 301)"
7,Numeric features,300
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.8747,0.8726,0.9750,0.8852,0.9279,0.4575,0.4911,19.6640
lr,Logistic Regression,0.8653,0.8542,0.9711,0.8788,0.9226,0.4115,0.4449,6.0200
svm,SVM - Linear Kernel,0.8611,0.0000,0.9527,0.8880,0.9188,0.4348,0.4569,4.6340
rf,Random Forest Classifier,0.8574,0.8289,0.9889,0.8598,0.9198,0.3029,0.3811,115.3700
lda,Linear Discriminant Analysis,0.8563,0.8433,0.9740,0.8683,0.9181,0.3465,0.3898,7.1670
qda,Quadratic Discriminant Analysis,0.8528,0.8178,0.9352,0.8921,0.9131,0.4328,0.4381,6.2190
et,Extra Trees Classifier,0.8489,0.8290,0.9934,0.8494,0.9158,0.2209,0.3197,36.5280
ada,Ada Boost Classifier,0.8487,0.8183,0.9563,0.8729,0.9127,0.3536,0.3757,116.8430
ridge,Ridge Classifier,0.8477,0.0000,0.9849,0.8535,0.9145,0.2479,0.3187,0.6790
dummy,Dummy Classifier,0.8272,0.5000,1.0000,0.8272,0.9054,0.0000,0.0000,0.4390


Processing:   0%|          | 0/53 [00:00<?, ?it/s]

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=8704, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

Com base nos dados do PyCaret o melhor algoritmo de classificação é o *Light Gradient Boosting Machine* o algoritmo mais próximo a este no PySpark é o *gradient boosting*. O PySpark vai ser utilizado para criar o modelo de ML final porque o dataset é maior que memoria ram disponível.

### Criar o classificador final utilizando  o pacote ML do PySpark

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors,VectorUDT
convert_array_to_vector_udf= udf(lambda x:  Vectors.dense(list(x)),VectorUDT())
df_word2vec_mlib=df_word2vec.withColumn('word2vec_vector',convert_array_to_vector_udf(col('word2vec')))

df_train, df_test = df_word2vec_mlib.randomSplit(weights=[0.8,0.2], seed=200)

In [ ]:
from pyspark.ml.classification import GBTClassifier

gbc=GBTClassifier(featuresCol='word2vec_vector', labelCol='polarity')
model = gbc.fit(df_train)


In [ ]:
gb_predictions = model.transform(df_test)
gb_predictions.write.parquet('predictions_wordvec_test.parquet')
gb_predictions_train = model.transform(df_train)
gb_predictions_train.write.parquet('predictions_wordvec_train.parquet')


In [ ]:
gb_predictions=df_word2vec=spark.read.parquet('predictions_wordvec_test.parquet')
gb_predictions_train=df_word2vec=spark.read.parquet('predictions_wordvec_train.parquet')

#### Analisar qualidade das predições no conjunto de testes

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from sklearn.metrics import confusion_matrix
import pandas as pd

evaluator = BinaryClassificationEvaluator(labelCol='polarity',rawPredictionCol='probability')
auroc = evaluator.evaluate(gb_predictions, {evaluator.metricName: "areaUnderROC"})
print("Area under ROC Curve: {:.4f}".format(auroc))

y_true = gb_predictions.select(['polarity']).collect()
y_pred = gb_predictions.select(['prediction']).collect()


Area under ROC Curve: 0.8277


In [ ]:
Matriz de confusão

In [ ]:
pd.DataFrame(confusion_matrix(y_true,y_pred),columns=[['predito','predito'],['Polaridade 0','Polaridade 1']], index=[['real','real'],['Polaridade 0','Polaridade 1']])


predito             
                  Polaridade 0 Polaridade 1
real Polaridade 0        21097        60526
     Polaridade 1         8050       387173

Analisando a métricas pode-se dizer que o modelo possui um AUC de qualidade razoavel. A matriz de confusão mostra que o modelo possui muitos falsos positivos, isso quer dizer que deve-se considerar positivo apenas os elementos cujo o modelo retorna alta probabilidade de serem positivos.

# Criar um classificador de sentimentos usando redes neurais
## Definir data generator

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import MaxPool1D
from tensorflow.keras.layers import Embedding, Input,Concatenate
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import random
import tqdm

class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, df,max_ratings,batch_size,vocab_size,nwords, shuffle=True):
        self.__df = df
        self.__batch_size = batch_size
        self.__shuffle = shuffle
        self.__vocab_size=vocab_size
        self.__nwords=nwords
        self.__max_ratings=max_ratings

    def __len__(self):
        return int(np.floor(len(self.__df) / self.__batch_size))
    
    def get_vocab_size(self):
        return int(self.__vocab_size)
    
    def get_nwords(self):
        return int(self.__nwords)

    def get_nclasses(self):
        return int(self.__max_ratings)+1

    def __getitem__(self, index):
        df = self.__df[index*self.__batch_size:(index+1)*self.__batch_size]
        x=np.zeros((len(df),self.__nwords),dtype=np.int32)
        y=np.zeros((len(df),int(self.__max_ratings)+1),dtype=np.float)
        for i in range(len(df)):
            row=df[i]
            text=row[0]
            text_np = self.__data_generation(text)
            x[i,:]=text_np
            y[i,int(row[1])]=1
        return x, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.__shuffle == True:
            random.shuffle(self.__df)

    def __data_generation(self, text):
        
        text_enc=one_hot(text,self.__vocab_size)
        padded_reviews = pad_sequences([text_enc],maxlen=self.get_nwords(),padding='post')[0]
        return padded_reviews




## Definir modelo

In [ ]:
def create_model_cnn(datagen:DataGenerator):
       
    number_of_classes=datagen.get_nclasses()
    model = Sequential()
    deep_inputs = Input(shape=(datagen.get_nwords(),))
    emb=Embedding(input_dim=datagen.get_vocab_size(),output_dim=128)(deep_inputs)


    conv1=Conv1D(32,3,padding='same',strides=1,activation='relu')(emb)
    maxPool1=MaxPool1D(pool_size=2,strides=2)(conv1)
    drop1=Dropout(0.2)(maxPool1)
    
    conv2=Conv1D(32,2,padding='same',strides=1,activation='relu')(drop1)
    maxPool2=MaxPool1D(pool_size=2,strides=2)(conv2)
    drop2=Dropout(0.2)(maxPool2)
    
    conv3=Conv1D(32,2,padding='same',strides=1,activation='relu')(drop2)
    maxPool3=MaxPool1D(pool_size=2,strides=2)(conv3)
    drop3=Dropout(0.2)(maxPool3)

    conv4=Conv1D(32,2,padding='same',strides=1,activation='relu')(drop3)
    maxPool4=MaxPool1D(pool_size=2,strides=2)(conv4)
    drop4=Dropout(0.2)(maxPool4)

    conv5=Conv1D(32,2,padding='same',strides=1,activation='relu')(drop4)
    maxPool5=MaxPool1D(pool_size=2,strides=2)(conv5)
    drop5=Dropout(0.2)(maxPool5)

    conv6=Conv1D(32,2,padding='same',strides=1,activation='relu')(drop5)
    maxPool6=MaxPool1D(pool_size=2,strides=2)(conv6)
    drop6=Dropout(0.2)(maxPool6)
    
    fla=Flatten()(drop6)
    den1=Dense(2000, activation='relu')(fla)
    drop_den1=Dropout(0.2)(den1)
    den2=Dense(2000, activation='relu')(drop_den1)
    drop_den2=Dropout(0.2)(den2)
    out=Dense(number_of_classes, activation='softmax')(drop_den2)
    model=Model(inputs=deep_inputs, outputs=out)
    # choosing Adam optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    # we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
    
    
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                   optimizer=optimizer,
                   metrics='accuracy')
         
 
    return model


## Criar classificador utilizando rede neural CNN

In [ ]:
batch_size=64
vocab_size = 50000
datasets=get_dataset_of_train_and_test()
datagen_train=DataGenerator(datasets['train'],max_ratings=datasets['max_rating'],batch_size=batch_size,vocab_size= vocab_size,nwords=256)
datagen_test=DataGenerator(datasets['test'],max_ratings=datasets['max_rating'],batch_size=batch_size,vocab_size= vocab_size,nwords=256)
model = create_model_cnn(datagen_train)

<ipython-input-3-a36d32f94ca2>:11: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  data=get_dataset()


In [ ]:
number_of_epochs = 5
history = model.fit(datagen_train,epochs=number_of_epochs,validation_data=datagen_test)

Epoch 1/5


<ipython-input-4-84a33e699cac>:46: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y=np.zeros((len(df),int(self.__max_ratings)+1),dtype=np.float)


26098/26098 [==============================] - 402s 15ms/step - loss: 0.2420 - accuracy: 0.9071 - val_loss: 0.1941 - val_accuracy: 0.9278
Epoch 2/5
26098/26098 [==============================] - 400s 15ms/step - loss: 0.1881 - accuracy: 0.9303 - val_loss: 0.1843 - val_accuracy: 0.9320
Epoch 3/5
26098/26098 [==============================] - 401s 15ms/step - loss: 0.1737 - accuracy: 0.9363 - val_loss: 0.1825 - val_accuracy: 0.9327
Epoch 4/5
26098/26098 [==============================] - 399s 15ms/step - loss: 0.1628 - accuracy: 0.9410 - val_loss: 0.1800 - val_accuracy: 0.9339
Epoch 5/5
26098/26098 [==============================] - 399s 15ms/step - loss: 0.1542 - accuracy: 0.9447 - val_loss: 0.1794 - val_accuracy: 0.9338


### Analisar qualidade das predições no conjunto de testes

Área sobre a curva roc


---



In [54]:
from sklearn.metrics import roc_auc_score
y_test=[]
y_test_prob=[]

for i in tqdm.tqdm(range(datagen_test.__len__())):
    x,y=datagen_test.__getitem__(i)
    prob=model.predict(x,verbose=False)
    for j in range(len(prob)):    
        y_test.append(int(y[j][1]))
        y_test_prob.append(prob[j,1])
roc_auc_score(y_test, y_test_prob)

  0%|          | 0/11185 [00:00<?, ?it/s]<ipython-input-4-84a33e699cac>:46: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y=np.zeros((len(df),int(self.__max_ratings)+1),dtype=np.float)
100%|██████████| 11185/11185 [10:05<00:00, 18.46it/s]


0.9584214056583868

Matriz de confusão

In [56]:
from sklearn.metrics import confusion_matrix

y_pred=[]
for x in y_test_prob:
    if(x>0.5):
        y_pred.append(1)
    else:
        y_pred.append(0)

pd.DataFrame(confusion_matrix(y_test,y_pred),columns=[['predito','predito'],['Polaridade 0','Polaridade 1']], index=[['real','real'],['Polaridade 0','Polaridade 1']])

predito             
                  Polaridade 0 Polaridade 1
real Polaridade 0        89500        33068
     Polaridade 1        14331       578941

Analisando a métricas pode-se dizer que o modelo possui um AUC de boa qualidade,estando bem próximo de 1 que é o valor máximo. A matriz de confusão mostra que o modelo possui uma boa acurácia (93%) e com bom valores para precisão e recall.

# Conclusão

Este notebook fez uma comparação entre duas abordagems para fazer um classificador de sentimentos em textos. Na primeira abordagem foi utilizado *gradient boosting*  para criar um classificador que recebe textos pré-processados  por *word2vec sum*. A segunda abordagem utilizou uma rede neural CNN. Com base nos resultados obtidos a segunda abordagem apresentou melhor qualidade. 

Para um trabalho futuro seria intersse observar o custo computacional de ambos os modelos para a relização de predição de dados, algo importanto quando utiliza-se algoritmos de *machine learning* em ambientes de produção.

# Referencias bibliograficas

RAAIJMAKERS, S. Deep Learning for Natural Language Processing. [s.l.] Manning Publications Company, 2019. 

UDAY KAMATH et al. Deep Learning for NLP and Speech Recognition. Cham: Springer International Publishing, 2019. 